In [1]:
import numpy as np
import pandas as pd



In [2]:
# initialize fees
FEE_F = 0.003

# initialize basic arrays and dictionaries
q = {'ab': [0., 0.], 'ac': [0., 0.], 'bc': [0.,0.]}
swap_amt = [0.]*2

# Test swap method, or not
TEST_SWAP_CODE = True

In [3]:
# Initialize dataframe

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [200., 200.]
    q_['bc'] = [250., 250.]
    return q_

q = reset_reserves(q)

init_state = {
    'exch': ['0'],
    'tok0': [0.],
    'tok1': [0.],
    'res0in': [0.],
    'res1in': [0.],
    'res0out': [0.],
    'res1out': [0.]
}

ledger_df = pd.DataFrame(init_state)

print(ledger_df)


  exch  tok0  tok1  res0in  res1in  res0out  res1out
0    0   0.0   0.0     0.0     0.0      0.0      0.0


In [4]:
# Swap code
#     Follows the Uniswap code

def psi(x):
    assert x >= 0
    return 1/(1+x)

def phi(x):
    assert x >= 0
    assert x < 1
    return 1/(1-x)

def swap(data_):
    # data_ = { 'reserves': q['ab'], 'legs': [10., 0.], 'direction': 'forward', 'fee': FEE_F}
    f_ = data_['fee']
    d_ = data_['legs']
    r_ = data_['reserves']
    direction:str = data_['direction']

    assert r_[0] > 0
    assert r_[1] > 0

    # The in_leg has the non-zero entry
    if d_[0] > 0:
        assert d_[1] == 0
        in_leg = 0
        out_leg = 1
    else:
        assert d_[0] == 0
        in_leg = 1
        out_leg = 0

    q_in = r_[in_leg]
    q_out = r_[out_leg]

    if direction == 'backward':
        # backward swap:
        #     find the input that produces the output
        #     the output is known, so this is the "in_leg" which has a non-zero entry
        d_out = d_[in_leg]
        d_in = float((1 / (1 - f_)) * (q_in / q_out) * phi(d_out / q_out) * d_out)
        result = d_in
    elif direction == 'forward':
        # forward swap:
        #     find the output produced by the input
        #     the input is known, so this is the "in_leg" which has a non-zero entry
        d_in = d_[in_leg]
        d_in_haircut = (1 - f_) * d_in
        d_out = float((q_out / q_in) * psi(d_in_haircut / q_in) * d_in_haircut)
        d_[out_leg] = d_out
        result = d_out
    else:
        print('unknown direction')
        return

    r_[out_leg] -= d_out
    r_[in_leg]  += d_in
    return {'reserves_in': [q_in, q_out], 'reserves_out': r_, 'legs': [d_in, d_out], 'result': result }


# Tests
if TEST_SWAP_CODE:
    q = reset_reserves(q)

    # swap a->b
    reason = 'route1: a->b'
    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [10., 0.],
                  'direction': 'forward',
                  'fee': FEE_F})
    print(reason)
    print(swap_out)
    route1_b_tokens = swap_out['result']

    # swap c->b
    reason = 'cycle: c->b '
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                  'legs': [route1_b_tokens, 0.],
                  'direction': 'backward',
                  'fee': FEE_F})
    cycle_c1_tokens = swap_out['result']
    print(reason)
    print(swap_out)
    print(f"cycle_c1_tokens: {cycle_c1_tokens}")


route1: a->b
{'reserves_in': [100.0, 100.0], 'reserves_out': [110.0, 90.93389106119851], 'legs': [10.0, 9.066108938801491], 'result': 9.066108938801491}
cycle: c->b 
{'reserves_in': [250.0, 250.0], 'reserves_out': [259.4355645298258, 240.9338910611985], 'legs': [9.435564529825822, 9.066108938801491], 'result': 9.435564529825822}
cycle_c1_tokens: 9.435564529825822


In [104]:
def run_compare_test(q, a_):
    # Run two routes: a->b and a->c->b
    # Notice that a->c->b returns *more* b-tokens

    # a->b route1
    q = reset_reserves(q)

    exch = 'ab'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [a_, 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    route1_b_tokens = swap_out['result']
    print(f"route1_a_tokens: {a_}")
    print(f"route1_b_tokens: {route1_b_tokens}")

    q_a_post_swap = q[exch][0]
    q_b_post_swap = q[exch][1]

    # a->c->b route2
    q = reset_reserves(q)

    # swap a->c
    exch = 'ac'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [a_, 0.],
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    route2_c_tokens = swap_out['result']
    print(f"route1_a_tokens: {a_}")
    print(f"route2_c_tokens: {route2_c_tokens}")

    # swap c->b
    exch = 'bc'
    swap_out = swap({ 'reserves': q[exch],
                      'legs': [0., route2_c_tokens], # note: incoming leg is c-tokens, which is index 1
                      'direction': 'forward',
                      'fee': FEE_F})
    ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
    route2_b_tokens = swap_out['result']
    print(f"route2_b_tokens: {route2_b_tokens}")

    b_token_difference = route2_b_tokens - route1_b_tokens
    print(f"Difference b-tokens route2 minus route1: {b_token_difference}")

    # Return the b-tokens
    return route1_b_tokens, route2_b_tokens, route2_c_tokens,q_a_post_swap, q_b_post_swap

In [105]:

def run_cycle_test(q, cycle, hinge_tokens, cycle_c1_tokens):

    print(f"Cycle: {cycle}")


    if cycle == "c1->b->a->c2":
        print(f"cycle_c1_tokens: {cycle_c1_tokens}")

        # swap b->a
        exch = 'ab'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [0., hinge_tokens],
                          'direction': 'backward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_a_tokens = swap_out['result']
        print(f"cycle_a_tokens: {cycle_a_tokens}")

        # swap a->c
        exch = 'ac'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [cycle_a_tokens, 0.],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_c2_tokens = swap_out['result']
        print(f"cycle_c2_tokens: {cycle_c2_tokens}")

        print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")

    else:
    # Run cycle: c1->a->b->c2

        # backward swap b->c
        exch = 'bc'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [hinge_tokens, 0.],
                          'direction': 'backward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_c2_tokens = swap_out['result']
        print(f"cycle_c2_tokens: {cycle_c2_tokens}")

        print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")



In [120]:
def run_cycles(q, cycle, cycle_c1_tokens):


    if cycle == "c1->b->a->c2":
        print(f"Cycle: {cycle}")

        print(f"cycle_c1_tokens: {cycle_c1_tokens}")

        # swap c-->b
        exch = 'bc'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [0., cycle_c1_tokens],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_b_tokens = swap_out['result']
        print(f"cycle_b_tokens: {cycle_b_tokens}")

        # swap b-->a
        exch = 'ab'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [0., cycle_b_tokens],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_a_tokens = swap_out['result']
        print(f"cycle_a_tokens: {cycle_a_tokens}")


        # swap a-->c
        exch = 'ac'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [cycle_a_tokens, 0.],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_c2_tokens = swap_out['result']
        print(f"cycle_c2_tokens: {cycle_c2_tokens}")


        print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")


    if cycle == "c1->a->b->c2":
        print(f"Cycle: {cycle}")

        print (f"cycle_c1_tokens: {cycle_c1_tokens}")

        # swap c-->a
        exch = 'ac'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [0., cycle_c1_tokens],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_a_tokens = swap_out['result']
        print(f"cycle_a_tokens: {cycle_a_tokens}")

        # swap a-->b
        exch = 'ab'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [cycle_a_tokens, 0.],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_b_tokens = swap_out['result']
        print(f"cycle_b_tokens: {cycle_b_tokens}")


        # swap b-->c
        exch = 'bc'
        swap_out = swap({ 'reserves': q[exch],
                          'legs': [cycle_b_tokens, 0.],
                          'direction': 'forward',
                          'fee': FEE_F})
        ledger_df.loc[len(ledger_df)] = [exch] + swap_out['legs'] + swap_out['reserves_in'] + swap_out['reserves_out']
        cycle_c2_tokens = swap_out['result']
        print(f"cycle_c2_tokens: {cycle_c2_tokens}")


        print(f"Difference c2 minus c1: {cycle_c2_tokens - cycle_c1_tokens}")




In [121]:
#   Liquidity Effects
#   The route using more liquidity has better prices

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [25000., 25000.]
    q_['bc'] = [25000., 25000.]
    return q_

q = reset_reserves(q)

nrow = len(ledger_df)
a_tokens = 10.0
route1_b_tokens, route2_b_tokens, route2_c_tokens, _, _ = run_compare_test(q, a_tokens)
if route2_b_tokens - route1_b_tokens > 0:
    cycle = "c1->b->a->c2"
    hinge_tokens = route2_b_tokens
else:
    cycle = "c1->a->b->c2"
    hinge_tokens = route1_b_tokens

q = reset_reserves(q)
run_cycle_test(q, cycle, hinge_tokens, route2_c_tokens  )
print(ledger_df[nrow:])

route1_a_tokens: 10.0
route1_b_tokens: 9.066108938801491
route1_a_tokens: 10.0
route2_c_tokens: 9.966025549011055
route2_b_tokens: 9.932179976111177
Difference b-tokens route2 minus route1: 0.8660710373096858
Cycle: c1->b->a->c2
cycle_c1_tokens: 9.966025549011055
cycle_a_tokens: 11.060627615937445
cycle_c2_tokens: 11.022583695348077
Difference c2 minus c1: 1.0565581463370215
    exch       tok0       tok1   res0in   res1in       res0out       res1out
292   ab  10.000000   9.066109    100.0    100.0    110.000000     90.933891
293   ac  10.000000   9.966026  25000.0  25000.0  25010.000000  24990.033974
294   bc   9.966026   9.932180  25000.0  25000.0  24990.067820  25009.966026
295   ab  11.060628   9.932180    100.0    100.0     90.067820    111.060628
296   ac  11.060628  11.022584  25000.0  25000.0  25011.060628  24988.977416


In [124]:
# Post-swap reserve effect
#   Test cycle arbitrage after a trade on more expensive exchange

def reset_reserves(q_:dict):
    q_['ab'] = [100., 100.]
    q_['ac'] = [25000., 25000.]
    q_['bc'] = [25000., 25000.]
    return q_

q = reset_reserves(q)

nrow = len(ledger_df)
a_tokens = 10.0
_, _, route2_c_tokens, q_a_post_swap, q_b_post_swap = run_compare_test(q, a_tokens)

# run cycle on the post-trade reserves q
q = reset_reserves(q)
q['ab'] = [q_a_post_swap, q_b_post_swap]

print(q)

cycle = "c1->b->a->c2"

run_cycles(q, cycle, route2_c_tokens  )
print(ledger_df[nrow:])

route1_a_tokens: 10.0
route1_b_tokens: 9.066108938801491
route1_a_tokens: 10.0
route2_c_tokens: 9.966025549011055
route2_b_tokens: 9.932179976111177
Difference b-tokens route2 minus route1: 0.8660710373096858
{'ab': [110.0, 90.93389106119851], 'ac': [25000.0, 25000.0], 'bc': [25000.0, 25000.0]}
Cycle: c1->b->a->c2
cycle_c1_tokens: 9.966025549011055
cycle_b_tokens: 9.932179976111177
cycle_a_tokens: 10.802285025002588
cycle_c2_tokens: 10.765240556756932
Difference c2 minus c1: 0.7992150077458771
    exch       tok0       tok1        res0in   res1in       res0out  \
309   ab  10.000000   9.066109    100.000000    100.0    110.000000   
310   ac  10.000000   9.966026  25000.000000  25000.0  25010.000000   
311   bc   9.966026   9.932180  25000.000000  25000.0  24990.067820   
312   bc   9.966026   9.932180  25000.000000  25000.0  24990.067820   
313   ab   9.932180  10.802285     90.933891    110.0     99.197715   
314   ac  10.802285  10.765241  25000.000000  25000.0  25010.802285   

   